In [ ]:
# Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, EfficientNetB0, VGG16
from tensorflow.keras.models import Model
import os
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from itertools import cycle
from sklearn.metrics import roc_curve, auc, roc_auc_score, RocCurveDisplay
from sklearn.preprocessing import label_binarize


import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from keras_tuner import HyperParameters 


# ----------- CONSTANTS ----------------

# define directory structure
TRAIN_DIR = "PROCESSED_DATA/TRAINING_DATA/TRAINING_AUGMENTED_DATA"
VALID_DIR = "PROCESSED_DATA/VALIDATION_DATA/"
TEST_DIR = "PROCESSED_DATA/TEST_DATA/"

# Image Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
NORMALIZE_FLAG = True
NO_FRILLS_DATAGEN = ImageDataGenerator()
NORM_DATAGEN = ImageDataGenerator(rescale=1./255)

TRAIN_GENERATOR = load_data(TRAIN_DIR)
VAL_GENERATOR = load_data(VALID_DIR)
TEST_GENERATOR = load_data(TEST_DIR,shuffle_flag=False)
hp = HyperParameters()


def load_data(directory,shuffle_flag=True):
    '''
    Param: 
        - directory - str, 
        - shuffle_flag - boolean, introduces constrolled stochasticity
    '''
    if NORMALIZE_FLAG == True:
        generator = NORM_DATAGEN.flow_from_directory(
        directory,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',shuffle=shuffle_flag)
        return generator
    else:
        generator = NO_FRILLS_DATAGEN.flow_from_directory(
        directory,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',shuffle=shuffle_flag)
        return generator

    
# MODEL BUILDING Functions -------------------

def build_tunable_cnn(hp):
    model = Sequential([
        Input(shape=(224, 224, 3)),
        Conv2D(32, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu', padding='same'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(hp.Float("dropout", min_value=0.2, max_value=0.5, step=0.1)), #tune dropout
        Dense(TRAIN_GENERATOR.num_classes, activation='softmax')
    ])
    
    # tune learning rate
    learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-2, sampling="log")
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model
    
def build_best_model():
    
    # Define the Bayesian tuner
    tuner = kt.BayesianOptimization(
        build_tunable_cnn,
        objective='val_accuracy',  # tune by improving validation accuracy
        max_trials=20,  # num different hp combos to try
        executions_per_trial=1,  # run each model once
        directory='bayesian_tuning',
        project_name='lr_and_drop_tuning'
    )
    
    # random search hp combos
    tuner.search(TRAIN_GENERATOR, validation_data=VAL_GENERATOR, epochs=1,batch_size = hp.Int(name='batch_size', min_value=16, max_value=32, step=16)
)
    
    # get best hps
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    # save them
    best_hps_dict = {'best_lr': best_hps.get('lr'),
                    'best_dropout': best_hps.get('dropout'),
                    'best_batch_size': best_hps.get('batch_size')}
    
    # make final model with the best drop out, learning rate and batch size
    best_model = tuner.hypermodel.build(best_hps)
    best_model_training_history = best_model.fit(TRAIN_GENERATOR, validation_data=VAL_GENERATOR, epochs=1, batch_size=best_hps.get('batch_size'))

    return best_hps_dict, best_model, best_model_training_history


def evaluate_model(model, filename = "best_model.h5"):
    '''
    Saves model to h5 file, returns test accuracy loss and test accuracy
    '''
    # evaluate on test data
    test_loss, test_acc = model.evaluate(TEST_GENERATOR)
    print(f"Test Accuracy: {test_acc:.4f}")
    
    # save to file
    model.save(filename)

    return test_loss, test_acc


def main():

    best_hps_dict, best_model, best_model_training_history = build_best_model()
    
    test_loss, test_acc = evaluate_model(best_model)
    print(f'test_loss{test_loss}, test_acc{test_acc}')
    
if __name__ == "__main__":
    main()


Trial 12 Complete [00h 01m 12s]
val_accuracy: 0.2863636314868927

Best val_accuracy So Far: 0.31590908765792847
Total elapsed time: 00h 15m 04s

Search: Running Trial #13

Value             |Best Value So Far |Hyperparameter
0.3               |0.3               |dropout
0.00036591        |0.00032041        |lr

 24/208 ━━━━━━━━━━━━━━━━━━━━ 1:00 330ms/step - accuracy: 0.1129 - loss: 2.5198